In [1]:
import pandas as pd
import numpy as np
import os
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import matplotlib.pyplot as plt


In [2]:
path = os.path.join(os.path.dirname(os.getcwd()),'raw_data')
df = pd.read_csv(os.path.join(path,'bbc_final_df.csv'))

In [3]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df = df[df['stars']!='n']

In [ ]:
df = df.sort_values(by='stars', ascending=False).reset_index(drop=True)

In [ ]:
df

In [ ]:
df1 = df[df['preference']=='american']

In [ ]:
df1.reset_index(drop=True, inplace=True)

In [ ]:
df1

In [ ]:
f = pd.DataFrame(df1.iloc[0]).T

In [ ]:
f

In [ ]:
d = pd.concat([df,f])

In [ ]:
d

In [4]:
def model_specific_preprocessing(tokenized_sentence):
    stop_words = set(stopwords.words('english'))
    no_stopwords = [w for w in tokenized_sentence if not w in stop_words]
    verb_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "v") for tok in no_stopwords]
    noun_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "n")for tok in verb_lemmatized]
    ingredient_string = ' '.join(noun_lemmatized)
    return ingredient_string

def ingredients_list(sample_txt):
    elements = []
    for punc in string.punctuation:
        sample_txt = sample_txt.replace(punc,'')
    for word in sample_txt.split():
        elements.append(str(word))
    return elements

In [7]:
def k_means(bbc_final_df, clusters=75, min_df=0.00001, max_df=0.3):
    """Returns the df inputted with an additional column with their respective cluster.
    Ensure that the ingredients columns is a string of a list called 'final_ingredients'.
    I.E row 1, col ingredient is '[bread,cheese,pasta,onions]'"""

    # Step 1: Filter out recipes with no names & remove duplicates, keeping dietary info
    filtered_df = bbc_final_df[bbc_final_df['recipe_title']!='n']
    grouped_df = filtered_df.groupby('recipe_title')['dietary'].apply(lambda x: ', '.join(x)).reset_index().rename(columns = {'dietary':'combined'})
    # grouped_df = filtered_df.groupby('recipe_title').sum()[['dietary']].reset_index()
    # grouped_df.rename(columns = {'dietary':'combined'}, inplace=True)
    merged_df = grouped_df.merge(filtered_df, how='left',on='recipe_title')
    #dropped_df = merged_df.drop(columns = 'dietary')
    df = merged_df.drop_duplicates('recipe_title')

    # Step 2: Convert the final ingredients into a "sentence" for vectorising
    df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
    df['clean_text']=df['ingredients_list'].map(lambda x: model_specific_preprocessing(x))

    # Step 3: Convert ingredients sentence into a bag-of-words representation
    vectorizer = CountVectorizer(min_df=min_df, max_df = max_df)
    counted_words = pd.DataFrame(vectorizer.fit_transform(df['clean_text']).toarray(), columns = vectorizer.get_feature_names_out())

    # Step 4: Apply KMeans clustering
    kmeans = KMeans(n_clusters=clusters, random_state=0)
    df['cluster'] = kmeans.fit_predict(counted_words)

    return df

In [8]:
df2= k_means(df)

/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_89439/2566586226.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_89439/2566586226.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text']=df['ingredients_list'].map(lambda x: model_specific_preprocessing(x))
/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_89439/2566586226.py:25: SettingWith

In [13]:
df2.iloc[35]

recipe_title                                                  Ajwain puri
combined                                    vegetarian, vegan, dairy-free
dietary                                                        vegetarian
stars                                                                   n
prep_times                                                             20
cooking_times                                                          20
review_count                                                            n
difficulty_level                                                        n
servings                                                         Serves 4
description             Enjoy these flatbread as part on an Indian fea...
specific_ingredients    300g ,chapatti flour, ½ tsp ,ajwain,, or cumin...
ingredients                                               sunflower oil, 
preference                                                         indian
final_ingredients                     

In [15]:
diet_df= (df2[df2.combined.str.contains('vegan')])

In [17]:
diet_df

,recipe_title,combined,dietary,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients,ingredients_list,clean_text,cluster
5,10-minute stir-fry,vegan,vegan,4.6,5,5,23.0,Easy,Serves 4,Grill a fillet of fish or chicken and serve wi...,"1 tbsp ,vegetable oil, 1 ,red chilli,, sliced ...","red chilli, garlic clove, soy sauce,",asian,"['chilli', 'garlic', 'clove', 'soy', 'sauce']","[chilli, garlic, clove, soy, sauce]",chilli garlic cleave soy sauce,11
20,5-ingredient falafel,"vegetarian, vegan, gluten-free, healthy",vegetarian,2.0,10,10,22.0,Easy,Serves 4-5 (makes 20),Make falafel with just a handful of storecupbo...,"125g ,dried chickpeas, ½large ,onion,, roughly...","onion, garlic cloves, olive oil,",middle-eastern,"['onion', 'garlic', 'cloves', 'olive', 'oil']","[onion, garlic, cloves, olive, oil]",onion garlic clove olive oil,1
39,Ajo blanco,"vegetarian, vegan, gluten-free",vegetarian,4.0,5,5,2.0,Easy,as a starter,"Miriam's easy, chilled almond soup makes a ref...","200g ,blanched almonds, 50ml ,extra virgin oli...","blanched almonds, extra virgin olive oil, garl...",spanish,"['almonds', 'olive', 'oil', 'garlic', 'clove']","[almonds, olive, oil, garlic, clove]",almond olive oil garlic cleave,72
42,Ajwain puri,"vegetarian, vegan, dairy-free",vegetarian,n,20,20,n,n,Serves 4,Enjoy these flatbread as part on an Indian fea...,"300g ,chapatti flour, ½ tsp ,ajwain,, or cumin...","sunflower oil,",indian,"['sunflower', 'oil']","[sunflower, oil]",sunflower oil,64
82,Aloo chaat,"vegan, gluten-free, egg-free",vegan,n,15,30,n,Easy,Serves 4-6,Enjoy a popular Indian festival snack. These l...,"750g ,potatoes, (Maris Piper or King Edward),...","potatoes, red onion, green chillies, ground cu...",indian,"['potato', 'onion', 'green', 'chillies', 'cumi...","[potato, onion, green, chillies, cumin, kashmi...",potato onion green chilli cumin kashmiri chill...,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10590,Wilted spinach with nutmeg & garlic,"vegetarian, vegan, gluten-free, healthy",vegetarian,4.7,5,5,3.0,Easy,Serves 4,Throw a bag of baby spinach in a wok with some...,"1-2 tbsp ,olive oil, 4 ,garlic cloves,, choppe...","olive oil, garlic cloves, bag baby spinach, nu...",british,"['olive', 'oil', 'garlic', 'cloves', 'spinach'...","[olive, oil, garlic, cloves, spinach, nutmeg]",olive oil garlic clove spinach nutmeg,1
10611,Winter vegetable caponata,"vegan, gluten-free, dairy-free, egg-free",vegan,3.3,15,45,4.0,Easy,Serves 2,Roast squash and turnips to give a winter veg ...,"200g ,squash,, peeled and cubed, 250g ,turnips...","olive oil, leeks, garlic cloves, balsamic vine...",italian,"['olive', 'oil', 'leek', 'garlic', 'cloves', '...","[olive, oil, leek, garlic, cloves, balsamic, c...",olive oil leek garlic clove balsamic caper oli...,29
10629,Yakitori corn pops,"vegetarian, vegan",vegetarian,4.3,15,20,3.0,Easy,Makes 8-12 depending on the size of your cobs,Try a new way of serving sweetcorn - skewer sl...,"2 ,fresh corn, cobs, leaves and threads peele...","fresh corn, sunflower oil,",japanese,"['corn', 'sunflower', 'oil']","[corn, sunflower, oil]",corn sunflower oil,64
10643,Za'atar,"vegetarian, vegan, gluten-free",vegetarian,4.8,5,2,6.0,Easy,makes 8,A fragrant Middle Eastern mix that will instan...,"2 tbsp ,cumin, 2 tbsp ,sesame seeds, 2 tbsp ,d...","cumin, dried oregano, sumac,",middle-eastern,"['cumin', 'oregano', 'sumac']","[cumin, oregano, sumac]",cumin oregano sumac,49


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
ingredients_cluster = df1[df1['cluster']==2][['recipe_title','clean_text','cluster']]


In [ ]:
ingredients_cluster = ingredients_cluster.reset_index(drop=True)

In [ ]:
ingredients_cluster_list = ingredients_cluster.clean_text.tolist()
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(ingredients_cluster_list)
similarity = cosine_similarity(vectors)

In [ ]:
similar= pd.DataFrame(similarity)

In [ ]:
similar

In [ ]:
#def similarity_cal(index):
similar[0].sort_values(ascending=False)

In [ ]:
ingredients_cluster['sim']=similar[0]

In [ ]:
ingredients_cluster.sort_values(by='sim', ascending=False)

In [ ]:
ingredients_cluster.head(3).clean_text

In [ ]:
p= pd.DataFrame(ingredients_cluster)

In [ ]:
p =p.sort_values(by='sim', ascending=False)

In [ ]:
p.head(15)

In [ ]:
def ingredients_list(sample_txt):
    elements = []
    for punc in string.punctuation:
        sample_txt = sample_txt.replace(punc,'')
    for word in sample_txt.split():
        elements.append(str(word))
    return elements

def remove_most_common_ings(ing_list):
    too_common = ['oil', 'butter', 'salt', 'clove', 'cloves']
    no_stopwords = [w for w in ing_list if not w in too_common]
    return ' '.join(no_stopwords)

In [ ]:
df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
df['temp_nocommon'] = df['ingredients_list'].apply(remove_most_common_ings)
df

In [ ]:
processed_df = pd.read_csv('/Users/camillemolen/code/mfaruki/replenish_frontend/raw_data/model_df.csv')


In [ ]:
select = 3

In [ ]:
pd.DataFrame(cuisine_counts).reset_index()

In [ ]:
cluster_counts = processed_df.groupby('cluster')['recipe_title'].count().reset_index()
cluster_counts.rename(columns={'recipe_title': 'Number of Recipes'}, inplace=True)

cluster_df = processed_df[processed_df['cluster'] == select]
cuisine_counts = cluster_df['preference'].value_counts()

plt.figure(figsize=(18, 6))
plt.bar(x=cuisine_counts.index, height=cuisine_counts.values)
plt.xlabel('Preference')
plt.ylabel('Number of Recipes')
plt.title(f'Cluster {select} - Preference vs. Number of Recipes')
plt.xticks(rotation=45)
plt.show()